# PCA-LDA

Given a collection of of n m-dimensional samples stored in the columns of matrix $X = [x_1 , . . . , x_n ]$. We also assume that data are centered. Otherwise, we can always substract the mean $m = \frac{1}{n} \sum_{i=1}^{n}x_i$.

An efficient way of doing this is:

\begin{align*}
X − M = X \left(I − \frac{1}{n}1_n1_n^T \right)
\end{align*}

## Statistical perspective
We want to find the vector $w$ such that the variance of the projected features $y_i = w_i^Tx_i$ is maximised.The variance can be expressed:

\begin{align*}
\sigma_y^2 = \frac{1}{n} \sum_{i=1}^{n}(y_i - \mu_y)
\end{align*}

Where $\mu_y$ is the mean. Since we supposed the data are centered, $\mu_y = 0$.

The maximiser is:

\begin{align*}
w^* & = \mathrm{argmax}_{w} \frac{1}{2n} \sum_{i=1}^{n}(w^Tx_i)^2 = \mathrm{argmax}_{w}  \frac{1}{2n} \sum_{i=1}^{n}w^Tx_i x_i^Tw\\
& = \mathrm{argmax}_{w} \frac{1}{2} w^T\frac{XX^T}{n}w  = \mathrm{argmax}_{w} \frac{1}{2} w^T S_tw  
\end{align*}

Where $S_t = \frac{XX^T}{n}$ is called the covariance matrix (or total-scatter matrix). We add the following constraint not to end up with a trivial solution $w=\infty$.

\begin{align*}
w^*  &= \mathrm{argmax}_{w} \frac{1}{2} w^T S_tw \\
\text{subject to } &w^Tw=1
\end{align*}

The corresponding Lagrangian is:

\begin{align*}
L(w, \lambda) = \frac{1}{2}w^TS_tw - \lambda(w^Tw-1)
\end{align*}

Taking the derivative:
\begin{align*}
\frac{ \partial L}{\partial w} = S_tw - \lambda w = 0
\end{align*}

This means $w$ is an eigenvector of $S_t$, and $\lambda$ the corresponding eigenvalue.